In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
df_wind = pd.read_csv(f'{os.getcwd()}\..\data\wind.csv', index_col=0)

df_wind = df_wind.drop(columns=['SS_Price', 'boa_MWh', 'DA_Price', 'Wind_MWh_credit', 'Solar_MWh_credit', 'Solar_MW', 'Solar_capacity_mwp', 'Solar_installedcapacity_mwp', 'dtm'])
df_wind['reference_time'] = pd.to_datetime(df_wind['reference_time'])
df_wind['valid_time'] = pd.to_datetime(df_wind['valid_time'])
df_latest = df_wind.groupby('valid_time').tail(1)

# Calculation using native features

In [3]:
df_latest['WindSpeed_avg'] = (df_latest['WindSpeed_dwd'] + df_latest['WindSpeed_ncep']) / 2
df_latest['Temperature_avg'] = (df_latest['Temperature_dwd'] + df_latest['Temperature_ncep']) / 2
df_latest['RelativeHumidity_avg'] = (df_latest['RelativeHumidity_dwd'] + df_latest['RelativeHumidity_ncep']) / 2

df_latest = df_latest.drop(columns=['WindSpeed_dwd', 'WindSpeed_ncep', 'Temperature_dwd', 'Temperature_ncep', 'RelativeHumidity_dwd', 'RelativeHumidity_ncep'])

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_39164\2943781978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest['WindSpeed_avg'] = (df_latest['WindSpeed_dwd'] + df_latest['WindSpeed_ncep']) / 2
C:\Users\t-ehm\AppData\Local\Temp\ipykernel_39164\2943781978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest['Temperature_avg'] = (df_latest['Temperature_dwd'] + df_latest['Temperature_ncep']) / 2
C:\Users\t-ehm\AppData\Local\Temp\ipykernel_39164\2943781978.py:3: SettingWithCopyWarning: 
A va

## Calculating Air denisity at each datapoint

In [4]:
# Constants
R_d = 287.05  # Specific gas constant for dry air (J/(kg·K))
R_v = 461.5   # Specific gas constant for water vapor (J/(kg·K))
p = 101325    # Standard atmospheric pressure in Pa

# Assuming df_latest is your original DataFrame and contains 'Temperature_dwd', 'RelativeHumidity_dwd', 'WindSpeed_dwd'
# Convert temperature from Celsius to Kelvin
df_latest['Temperature_K'] = df_latest['Temperature_avg'] + 273.15

# Calculate saturation vapor pressure (using temperature in Celsius), Tetens formula
e_s = 0.61078 * np.exp((17.27 * df_latest['Temperature_avg']) / (df_latest['Temperature_avg'] + 237.3))

# in pa
e_s = 1000 * e_s

# Calculate actual vapor pressure
e = df_latest['RelativeHumidity_avg'] / 100 * e_s

# Calculate air density (ρ) in kg/m³
df_latest['AirDensity'] = (p - e) / (R_d * df_latest['Temperature_K']) + (e / (R_v * df_latest['Temperature_K']))

In [11]:
# Turbine stats
rotor_diameter = 154  # in meters
approximated_total_efficiency = 0.46  # 46% efficiency
minimum_wind_speed = 3  # in m/s
maximum_wind_speed_for_power_curve = 11  # in m/s
maximum_wind_speed_for_operation = 25  # in m/s
rotor_area = np.pi * (rotor_diameter / 2) ** 2  # in m²
const_internal_friction_coefficient = 0.5 * 1.225 * np.pi * 77**2 * 3**3 * 0.46 * 174 / 1000000

# Calculating the Generated power
df_latest['WindPower'] = 0.5 * df_latest['AirDensity'] * rotor_area * df_latest['WindSpeed_avg'] ** 3 * 174 / 1000000
df_latest['UsableWindPower'] = np.minimum(df_latest['WindPower'], 0.5 * df_latest['AirDensity'] * rotor_area * maximum_wind_speed_for_power_curve ** 3 * 174 / 1000000) - const_internal_friction_coefficient
# depending on the wind speed, the power output is limited to the maximum power output of the turbine or 0
df_latest['PowerOutput'] = np.where((df_latest['WindSpeed_avg'] >= minimum_wind_speed) & (df_latest['WindSpeed_avg'] <= maximum_wind_speed_for_operation), df_latest['UsableWindPower'] * approximated_total_efficiency, 0)

In [12]:
# compare average lines in 0.5 windspeed bins of PowerOutput and Wind_MW
import plotly.express as px
import plotly.graph_objects as go

df_latest['WindSpeed_avg_bin'] = pd.cut(df_latest['WindSpeed_avg'], bins=np.arange(0, 26, 0.5))
df_grouped = df_latest.groupby('WindSpeed_avg_bin').mean().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_grouped['WindSpeed_avg'], y=df_grouped['PowerOutput'], mode='lines', name='PowerOutput'))
fig.add_trace(go.Scatter(x=df_grouped['WindSpeed_avg'], y=df_grouped['Wind_MW'], mode='lines', name='Wind_MW'))
fig.show()

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_39164\1216776015.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# Creating more advanced features, interpolating between lag and future features to obtain better forecasts over 30min

In [ ]:
import pandas as pd
import numpy as np

# Cyclical encoding for wind direction
df_wind['sin_WindDirection_dwd'] = np.sin(df_wind['WindDirection_dwd'] * np.pi / 180)
df_wind['cos_WindDirection_dwd'] = np.cos(df_wind['WindDirection_dwd'] * np.pi / 180)
df_wind['sin_WindDirection_ncep'] = np.sin(df_wind['WindDirection_ncep'] * np.pi / 180)
df_wind['cos_WindDirection_ncep'] = np.cos(df_wind['WindDirection_ncep'] * np.pi / 180)

# Lag features (e.g., 1 time step in the past, assuming 30-minute intervals)
df_wind['WindSpeed_dwd_lag_1'] = df_wind['WindSpeed_dwd'].shift(1)
df_wind['Temperature_dwd_lag_1'] = df_wind['Temperature_dwd'].shift(1)
df_wind['RelativeHumidity_dwd_lag_1'] = df_wind['RelativeHumidity_dwd'].shift(1)

# Difference features
df_wind['wind_speed_diff'] = df_wind['WindSpeed_dwd'] - df_wind['WindSpeed_ncep']
df_wind['temperature_diff'] = df_wind['Temperature_dwd'] - df_wind['Temperature_ncep']

# Interaction features
df_wind['wind_temp_interaction'] = df_wind['WindSpeed_dwd'] * df_wind['Temperature_dwd']
df_wind['humidity_wind_interaction'] = df_wind['WindSpeed_dwd'] * df_wind['RelativeHumidity_dwd']

# Rolling window features (e.g., rolling mean over 1 hour, 2 intervals for 30-minute steps)
df_wind['WindSpeed_dwd_rolling_mean_1h'] = df_wind['WindSpeed_dwd'].rolling(window=2).mean()
df_wind['Temperature_dwd_rolling_std_2h'] = df_wind['Temperature_dwd'].rolling(window=4).std()

# Drop rows with NaN values introduced by lagging or rolling (optional)
df_wind = df_wind.dropna()


In [ ]:
# Cyclical encoding for wind direction

# Average WindDirection
df_wind['WindDirection_avg'] = (df_wind['WindDirection_dwd'] + df_wind['WindDirection_ncep']) / 2
df_wind['WindDirection_100_avg'] =(df_wind['WindDirection:100_dwd'] + df_wind['WindDirection:100_ncep']) / 2
df_wind['sin_WindDirection_avg'] = np.sin(df_wind['WindDirection_avg'] * np.pi / 180)
df_wind['cos_WindDirection_avg'] = np.cos(df_wind['WindDirection_avg'] * np.pi / 180)
df_wind = df_wind.drop(columns=['WindDirection_dwd', 'WindDirection_ncep', 'WindDirection:100_dwd', 'WindDirection:100_ncep', 'WindDirection_avg', 'WindDirection_100_avg'])

# Average WindSpeed, Temperature, and RelativeHumidity, Lag features
df_wind['WindSpeed_avg'] = (df_wind['WindSpeed_dwd'] + df_wind['WindSpeed_ncep']) / 2
df_wind['Temperature_avg'] = (df_wind['Temperature_dwd'] + df_wind['Temperature_ncep']) / 2
df_wind['RelativeHumidity_avg'] = (df_wind['RelativeHumidity_dwd'] + df_wind['RelativeHumidity_ncep']) / 2
df_wind['WindSpeed_avg_lag_1'] = df_wind['WindSpeed_avg'].shift(1)
df_wind['WindSpeed_avg_lag_2'] = df_wind['WindSpeed_avg'].shift(2) 
# df_wind['WindSpeed_avg_lag_1_diff'] = df_wind['WindSpeed_avg'].shift(1) - df_wind['WindSpeed_avg']
# df_wind['WindSpeed_avg_lag_2_diff'] = df_wind['WindSpeed_avg'].shift(2) - df_wind['WindSpeed_avg']
df_wind['Temperature_avg_lag_1'] = df_wind['Temperature_avg'].shift(1)
df_wind['Temperature_avg_lag_2'] = df_wind['Temperature_avg'].shift(2)
# df_wind['Temperature_avg_lag_1_diff'] = df_wind['Temperature_avg'].shift(1) - df_wind['Temperature_avg']
# df_wind['Temperature_avg_lag_2_diff'] = df_wind['Temperature_avg'].shift(2) - df_wind['Temperature_avg']
df_wind['RelativeHumidity_avg_lag_1'] = df_wind['RelativeHumidity_avg'].shift(1)
df_wind['RelativeHumidity_avg_lag_2'] = df_wind['RelativeHumidity_avg'].shift(2)
# df_wind['RelativeHumidity_avg_lag_1_diff'] = df_wind['RelativeHumidity_avg'].shift(1) - df_wind['RelativeHumidity_avg']
# df_wind['RelativeHumidity_avg_lag_2_diff'] = df_wind['RelativeHumidity_avg'].shift(2) - df_wind['RelativeHumidity_avg']
df_wind = df_wind.drop(columns=['WindSpeed_dwd', 'WindSpeed_ncep', 'Temperature_dwd', 'Temperature_ncep', 'RelativeHumidity_dwd', 'RelativeHumidity_ncep'])

# Lag features (e.g., 1 time step in the past, assuming 30-minute intervals)
df_wind['WindSpeed_dwd_lag_1'] = df_wind['WindSpeed_dwd'].shift(1)
df_wind['Temperature_dwd_lag_1'] = df_wind['Temperature_dwd'].shift(1)
df_wind['RelativeHumidity_dwd_lag_1'] = df_wind['RelativeHumidity_dwd'].shift(1)

# Difference features
df_wind['wind_speed_diff'] = df_wind['WindSpeed_dwd'] - df_wind['WindSpeed_ncep']
df_wind['temperature_diff'] = df_wind['Temperature_dwd'] - df_wind['Temperature_ncep']

# Interaction features
df_wind['wind_temp_interaction'] = df_wind['WindSpeed_dwd'] * df_wind['Temperature_dwd']
df_wind['humidity_wind_interaction'] = df_wind['WindSpeed_dwd'] * df_wind['RelativeHumidity_dwd']

# Rolling window features (e.g., rolling mean over 1 hour, 2 intervals for 30-minute steps)
df_wind['WindSpeed_dwd_rolling_mean_1h'] = df_wind['WindSpeed_dwd'].rolling(window=2).mean()
df_wind['Temperature_dwd_rolling_std_2h'] = df_wind['Temperature_dwd'].rolling(window=4).std()

# Drop rows with NaN values introduced by lagging or rolling (optional)
df_wind = df_wind.dropna()